In [ ]:
%pip install pyarrow==15.0.2
%pip install datasets
%%capture
%pip install accelerate peft bitsandbytes transformers trl
%pip install -U bitsandbytes

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer



In [ ]:
df=pd.read_csv(r"/content/all about iphone.csv")

In [ ]:
df.head()

,Question,Answer,Formatted
0,How do I take a screenshot on an iPhone?,"To take a screenshot on an iPhone, press and h...",[INST] How do I take a screenshot on an iPhone...
1,How do I change my wallpaper on an iPhone?,"To change your wallpaper on an iPhone, go to S...",[INST] How do I change my wallpaper on an iPho...
2,How do I make a phone call on an iPhone?,"To make a phone call on an iPhone, open the Ph...",[INST] How do I make a phone call on an iPhone...
3,How do I send a text message on an iPhone?,"To send a text message on an iPhone, open the ...",[INST] How do I send a text message on an iPho...
4,How do I use Siri on an iPhone?,"To use Siri on an iPhone, press and hold the H...",[INST] How do I use Siri on an iPhone? [/INST]...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  1005 non-null   object
 1   Answer    1004 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [ ]:
def format_row(row):
  question=row["Question"]
  answer=row["Answer"]
  formatted_string=f"[INST] {question} [/INST] {answer}"
  return formatted_string

In [ ]:
df["Formatted"]=df.apply(format_row,axis=1)

In [ ]:
df["Formatted"]

,Formatted
0,[INST] How do I take a screenshot on an iPhone...
1,[INST] How do I change my wallpaper on an iPho...
2,[INST] How do I make a phone call on an iPhone...
3,[INST] How do I send a text message on an iPho...
4,[INST] How do I use Siri on an iPhone? [/INST]...
...,...
1000,[INST] How to clear cache on Safari on my Mac ...
1001,[INST] What to do if my AirPods Pro have low v...
1002,[INST] Can I use a wireless trackpad with my M...
1003,[INST] How to transfer files from my Mac to my...


In [ ]:
new_df=df.rename(columns={"Formatted":"Text"})

In [ ]:
new_df

,Question,Answer,Text
0,How do I take a screenshot on an iPhone?,"To take a screenshot on an iPhone, press and h...",[INST] How do I take a screenshot on an iPhone...
1,How do I change my wallpaper on an iPhone?,"To change your wallpaper on an iPhone, go to S...",[INST] How do I change my wallpaper on an iPho...
2,How do I make a phone call on an iPhone?,"To make a phone call on an iPhone, open the Ph...",[INST] How do I make a phone call on an iPhone...
3,How do I send a text message on an iPhone?,"To send a text message on an iPhone, open the ...",[INST] How do I send a text message on an iPho...
4,How do I use Siri on an iPhone?,"To use Siri on an iPhone, press and hold the H...",[INST] How do I use Siri on an iPhone? [/INST]...
...,...,...,...
1000,How to clear cache on Safari on my Mac for imp...,Go to Safari > Preferences > Privacy > Manage ...,[INST] How to clear cache on Safari on my Mac ...
1001,What to do if my AirPods Pro have low volume?,"Clean the earbuds, ensure they are correctly s...",[INST] What to do if my AirPods Pro have low v...
1002,Can I use a wireless trackpad with my Mac for ...,"Yes, you can use a wireless Bluetooth trackpad...",[INST] Can I use a wireless trackpad with my M...
1003,How to transfer files from my Mac to my iPhone...,"Use apps like Files, iCloud Drive, or third-pa...",[INST] How to transfer files from my Mac to my...


In [ ]:
new_df=new_df["Text"]

In [ ]:
new_df.head()

,Text
0,[INST] How do I take a screenshot on an iPhone...
1,[INST] How do I change my wallpaper on an iPho...
2,[INST] How do I make a phone call on an iPhone...
3,[INST] How do I send a text message on an iPho...
4,[INST] How do I use Siri on an iPhone? [/INST]...


In [ ]:
new_df.to_csv("Formatted_data.csv",index=False)

In [ ]:
%pip install pyarrow==15.0.2
from datasets import load_dataset
training_dataset=load_dataset("csv",data_files="Formatted_data.csv",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
training_dataset

Dataset({
    features: ['Text'],
    num_rows: 1005
})

In [ ]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model
new_model = "llama-2-7b-chat-iphone"

In [ ]:
compute_dtype = getattr(torch, "float16")


In [ ]:
quant_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    peft_config=peft_params,
    dataset_text_field="Text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1005 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
trainer.train()

Step,Training Loss
25,1.733700
50,1.070100
75,1.070900
100,0.936600
125,0.930000
150,0.880700
175,0.858800
200,0.930300
225,0.757500
250,0.850300


TrainOutput(global_step=252, training_loss=0.9984015633189489, metrics={'train_runtime': 705.0934, 'train_samples_per_second': 1.425, 'train_steps_per_second': 0.357, 'total_flos': 2325855005147136.0, 'train_loss': 0.9984015633189489, 'epoch': 1.0})

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('llama-2-7b-chat-iphone/tokenizer_config.json',
 'llama-2-7b-chat-iphone/special_tokens_map.json',
 'llama-2-7b-chat-iphone/tokenizer.model',
 'llama-2-7b-chat-iphone/added_tokens.json',
 'llama-2-7b-chat-iphone/tokenizer.json')

In [ ]:
prompt = "How do I set up parental controls on my iPhone?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>[INST] How do I set up parental controls on my iPhone? [/INST] Go to Settings > Screen Time > Content Restrictions to set up parental controls. You can set restrictions on content, apps, and more. You can also use Screen Time to monitor your child's screen time and set limits.

You can also use Family Sharing to manage your child's Apple ID and set up parental controls on their devices. Go to Settings > Family Sharing > Screen Time to set up parental controls.

You can also use the Find My app to locate your child's device and set up alerts to help you keep track of their whereabouts.

You can also use the Messages app to set up Messages restrictions, which can help you control the types of messages your child can send and receive.

You can also use the FaceTime app to set up FaceTime restrictions, which can help you control


In [ ]:
prompt = "How do I take a screenshot on an iPhone?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How do I take a screenshot on an iPhone? [/INST] Press and hold the Power button and the Volume Down button simultaneously. To take a screenshot of the entire screen, press and hold the Power button and the Home button at the same time. To take a screenshot of a specific app, press and hold the Power button and the Home button, then release the Power button. You will hear a camera shutter sound, and the screenshot will be saved to your Photos app.

Note: The steps for taking a screenshot may vary depending on the iPhone model and iOS version.
